# Tugas Besar Pemroseran Parallel Kelas B
## Analisis Data Games menggunakan MPI (reduce, filter), pyspark context (reduce, filter), dan sparksession (query) 

1. Rakasya Yoga Surya Prastama (F1D02310022)
2. Rakasya Yoga Surya Prastama (F1D02310022)
3. Rakasya Yoga Surya Prastama (F1D02310022)
4. Rakasya Yoga Surya Prastama (F1D02310022)

Link Kaggle untuk dataset: https://www.kaggle.com/datasets/jummyegg/rawg-game-dataset

In [ ]:
%pip install pyspark
%pip install mpi4py

Baris kode ini Menginstall 2 library yang akan digunakan, yaitu 'pyspark' dan juga 'mpi4py' dengan perintah '%pip install. 'pyspark' digunakan untuk menjalankan Spark di lingkungan Python, yang memungkinkan pemrosesan data besar secara paralel dan terdistribusi. Sedangkan 'mpi4py' adalah antarmuka Python untuk MPI (Message Passing Interface), yang digunakan untuk membagi dan mengelola tugas antar proses dalam pemrograman paralel manual. 

In [ ]:
from mpi4py import MPI
from pyspark.sql import SparkSession
import pandas as pd
import csv

Baris kode ini memuat semua library yang akan digunakan untuk projek ini. 'mpi4py.MPI' digunakan untuk komunikasi antar proses saat menggunakan 'MPI', sedangkan 'SparkSession' dari 'pyspark.sql' digunakan untuk membaca dan memproses data tabular seperti database. Library tambahan seperti 'pandas', 'numpy', dan 'csv' juga diimpor, kemungkinan untuk manipulasi data dan pembacaan file CSV di luar konteks Spark.

In [ ]:
import time
mulai_ukur = time.time()
akhir_ukur = time.time()

waktu_pengerjaan = akhir_ukur - mulai_ukur

print(waktu_pengerjaan)

Baris kode ini bertujuan mengukur durasi proses yang akan dijalankan. Namun, karena waktu mulai yaitu 'mulai_ukur' dan akhir yaitu 'akhir_ukur' dicatat tanpa proses di antaranya, nilai 'waktu_pengerjaan' akan mendekati nol. Jika nilai ditempatkan dengan benar di sekitar blok pemrosesan utama, pengukuran ini nantinya akan berguna untuk membandingkan kinerja MPI dan PySpark.

In [20]:
spark = SparkSession.builder.appName("games").getOrCreate()
data = spark.read.csv("game_info.csv", header=True, inferSchema=True)
data.show(10)


+------+--------------------+--------------------+----------+----------+-----+-------------------+--------------------+------+----------+--------+------------------+-------------+-----------------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+------------+----------------+------------------+-------------------+-------------------+--------------------+--------------------+
|    id|                slug|                name|metacritic|  released|  tba|            updated|             website|rating|rating_top|playtime|achievements_count|ratings_count|suggestions_count|game_series_count|reviews_count|           platforms|          developers|              genres|          publishers| esrb_rating|added_status_yet|added_status_owned|added_status_beaten|added_status_toplay|added_status_dropped|added_status_playing|
+------+--------------------+--------------------+----------+----------+-----+-------------------+------------

Pada baris kode ini 'SparkSession' dibuat dengan nama aplikasi "Game Sales Analysis", lalu digunakan untuk membaca file CSV 'game_info.csv'. File dibaca sebagai DataFrame dengan parameter 'header=True' agar baris pertama digunakan sebagai nama kolom dan 'inferSchema=True' agar tipe data dikenali otomatis. Kemudian, sepuluh baris pertama dari data ditampilkan untuk keperluan verifikasi.

In [ ]:
mpi = MPI.COMM_WORLD
size = mpi.Get_size()

data = pd.read_csv("game_info.csv")
chunks = [data.iloc[i::size].reset_index(drop=True) for i in range(size)]

scatter = mpi.scatter(chunks, root=0) # Scatter

Filter = scatter[scatter["metacritic"] > 80] # Filter dari metacritic di atas 80

Reduce = Filter["ratings_count"].sum() #menghitung jumlah ratings_count sesuii filter
total_Reduce = mpi.reduce(Reduce, op=MPI.SUM, root=0) # Reduce

print("Total ratings_count untuk metacritic > 80:", total_Reduce)

Total ratings_count untuk metacritic > 80: 427925


Pada baris Kode ini dengan Menggunakan mpi4py, data dibaca melalui 'pandas' dan dibagi menjadi beberapa potongan berdasarkan jumlah proses (size). Setiap proses mendapatkan bagian data untuk dianalisis secara paralel menggunakan scatter. Filter dilakukan untuk memilih data dengan nilai metacritic > 80, kemudian dilakukan reduce untuk menghitung total ratings_count, dan hasil akhirnya dikumpulkan di root process menggunakan mpi.reduce.

In [ ]:
sparkContext = spark.sparkContext
rdd = sparkContext.textFile("game_info.csv")

header = rdd.first()

def remove_header(row):
    return row != header

def parse_row(row):
    return next(csv.reader([row]))

def filter_metacritic(row):
    return row[3].isdigit() and int(row[3]) > 80 # 3 itu index kolom metacritic

def extract_ratings_count(row):
    if row[12].isdigit(): # 12 itu index kolom ratings_count
        return int(row[12])
    return 0

def add(a, b):
    return a + b

data = rdd.filter(remove_header).map(parse_row)

filtered = data.filter(filter_metacritic)

ratings = filtered.map(extract_ratings_count)

total = ratings.reduce(add)

print("Total ratings_count untuk metacritic > 80:", total)

Total ratings_count untuk metacritic > 80: 427925


Pada baris kode ini Data CSV dibaca sebagai RDD menggunakan 'SparkContext', dan 'header' dihapus dari 'data'. Setiap baris diubah menjadi list menggunakan parser CSV, kemudian difilter agar hanya menyisakan baris dengan metacritic > 80. Nilai 'ratings_count' dikonversi menjadi integer dan dijumlahkan menggunakan fungsi reduce. Hasil akhirnya adalah total 'ratings_count' dari data yang telah difilter.

In [37]:
# data = spark.read.csv("game_info.csv", header=True, inferSchema=True)
# data.createOrReplaceTempView("games")

result = spark.sql("""
    SELECT name, rating, metacritic, platforms, released, genres, publishers, developers
    FROM games
    WHERE 
        rating IS NOT NULL AND 
        metacritic IS NOT NULL AND
        TRY_CAST(rating AS FLOAT) IS NOT NULL AND 
        metacritic IS NOT NULL
    ORDER BY rating  DESC
    LIMIT 20
""")

print("Hasil Query:")
result.toPandas()

Hasil Query:


,name,rating,metacritic,platforms,released,genres,publishers,developers
0,Persona 5 Royal,4.81,95,PlayStation 4,2020-03-31,Adventure||RPG,SEGA||Atlus,Atlus
1,The Witcher 3: Game of the Year,4.77,92,Xbox One||PlayStation 4||Nintendo Switch||PC,2016-08-30,Adventure||RPG,CD PROJEKT RED,CD PROJEKT RED
2,The Witcher 3: Wild Hunt - Blood and Wine,4.75,92,PC||Xbox One||PlayStation 4,2016-05-30,RPG,CD PROJEKT RED,CD PROJEKT RED
3,The Witcher 3: Wild Hunt - Hearts of Stone,4.74,90,PC||Xbox One||PlayStation 4,2015-10-13,RPG,CD PROJEKT RED||cdp.pl,CD PROJEKT RED
4,Yakuza: Like a Dragon,4.73,84,PlayStation 5||Xbox Series S/X||Xbox One||PC||...,2020-01-16,Action||RPG,SEGA,SEGA||Ryu ga Gotoku Studio
5,Grand Theft Auto IV: Complete Edition,4.73,95,Xbox 360||PlayStation 3||PC,2010-10-27,Action||Adventure,Rockstar Games,Rockstar Toronto
6,The Last Of Us Remastered,4.7,95,PlayStation 4,2014-07-29,Action||Adventure,Sony Computer Entertainment,Naughty Dog
7,Dark Souls III: The Ringed City,4.7,83,PlayStation 4||Xbox One||PC,2017-03-28,Action||RPG,Bandai Namco Entertainment,FromSoftware
8,Kingdom Hearts II Final Mix,4.67,87,PlayStation 2||PlayStation 3||PlayStation 4,2007-03-29,Action||RPG,Square Enix,Square Enix
9,Drill Dozer (2005),4.67,81,Game Boy Advance||Wii U,2005-09-22,Action,Nintendo,Game Freak


Baris kode ini menggunakan fitur SQL dari Spark untuk menjalankan kueri terhadap DataFrame yang telah dibuat sebelumnya. Data difilter agar hanya menampilkan game yang memiliki nilai rating dan metacritic yang valid, lalu diurutkan berdasarkan rating secara menurun. Hasilnya dibatasi hanya 20 game teratas dan ditampilkan sebagai DataFrame 'pandas' agar lebih mudah dibaca.